---
# Team IF Statement's Checkpoint 1
Members: Logan, Matthew, Veena

In [0]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
# any number will do, as long as it is used consistently
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [0]:
import pandas as pd

# Load all datasets
dropout2017 = pd.read_csv("http://www.doe.virginia.gov/statistics_reports/research_data/data_files/dropouts/dropouts-2017-results.csv")
dropout2016 = pd.read_csv("http://www.doe.virginia.gov/statistics_reports/research_data/data_files/dropouts/dropouts-2016-results.csv")
dropout2015 = pd.read_csv("http://www.doe.virginia.gov/statistics_reports/research_data/data_files/dropouts/dropouts-2015-results.csv")
dropout2014 = pd.read_csv("http://www.doe.virginia.gov/statistics_reports/research_data/data_files/dropouts/dropouts-2014-results.csv")
dropout2013 = pd.read_csv("http://www.doe.virginia.gov/statistics_reports/research_data/data_files/dropouts/dropouts-2013-results.csv")

dropout = dropout2017.append([dropout2016, dropout2015, dropout2014, dropout2013], ignore_index=True)
dropout.head()

,SCHOOL_YEAR,LEVEL_CODE,DIV_NUM,DIV_NAME,SCH_NUM,SCH_NAME,FEDERAL_RACE_CODE,GENDER,GRADE_CODE,DISABILITY_FLAG,DISADVANTAGED_FLAG,LEP_FLAG,DROPOUT_CNT
0,2017-2018,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7541
1,2017-2018,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,5506
2,2017-2018,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,2035
3,2017-2018,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,3942
4,2017-2018,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,2914


In [0]:
dropout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100046 entries, 0 to 100045
Data columns (total 13 columns):
SCHOOL_YEAR           100046 non-null object
LEVEL_CODE            100046 non-null object
DIV_NUM               90036 non-null float64
DIV_NAME              90036 non-null object
SCH_NUM               42190 non-null float64
SCH_NAME              42190 non-null object
FEDERAL_RACE_CODE     48136 non-null float64
GENDER                47966 non-null object
GRADE_CODE            46429 non-null float64
DISABILITY_FLAG       48472 non-null object
DISADVANTAGED_FLAG    48139 non-null object
LEP_FLAG              50291 non-null object
DROPOUT_CNT           100046 non-null int64
dtypes: float64(4), int64(1), object(8)
memory usage: 9.9+ MB


In [0]:
dropout.columns

Index(['SCHOOL_YEAR', 'LEVEL_CODE', 'DIV_NUM', 'DIV_NAME', 'SCH_NUM',
       'SCH_NAME', 'FEDERAL_RACE_CODE', 'GENDER', 'GRADE_CODE',
       'DISABILITY_FLAG', 'DISADVANTAGED_FLAG', 'LEP_FLAG', 'DROPOUT_CNT'],
      dtype='object')

# Processing the data
We have a few problems with the data:


*    **Unecessary Fields**: A few of these fields are repetitive (DIV_NAME is eoncded in DIV_NUM), so we need to drop these.
*   **Categorical Variables**: We need to encode our categorical variables, but we do need to worry about number of features (we can't just one hot encode everything). Therefore, we will sparsely encode most features to keep feature dim small.
*    **NaN values**: The data uses NaN values as an actual value, so we need to have a difference between NaN and encoded values.

The following data process solves all of these problems. We can still tweak this process a lot (perhaps go from -1, 0, 1 encoding to 0, 1, 2, or maybe do everything with dense one hot encoding).



## Custom Transformer
We can use standard sklearn transformers for NaN values and encoding, but I want to encode the various flags on a -1, 0, 1 scale (I hoped this symmetry would keep the weight around zero. This isn't easily done with sklearn transformers, so this is a custom transformer than econdes the "Y, N, M, F" values appropriately (NaN is 0).

In [0]:
from sklearn.base import BaseEstimator, TransformerMixin

class replaceVals(BaseEstimator, TransformerMixin):
    def __init__(self):
        return
    def fit(self, X):
        return self 
    def transform(self, X):
      X[X == 'Y'] = 1
      X[X == 'N'] = -1
      X[X == 'M'] = 1
      X[X == 'F'] = -1
      
      return X

## The Pipeline

**Specify where to transform**: We need to tell our pipeline where to replace missing values and where to transform, and we can also leave out columns (which drops them).

In [0]:
# These are the columns to fill missing values
fill_cols = ['DIV_NUM', 'SCH_NUM', 'FEDERAL_RACE_CODE', 'GENDER',
       'GRADE_CODE', 'DISABILITY_FLAG', 'DISADVANTAGED_FLAG', 'LEP_FLAG', 'DROPOUT_CNT']

# We will only densely one hot encode Level Code
one_hot_cols = ["SCHOOL_YEAR", "LEVEL_CODE"]

**The Pipelines**

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Replacement pipeline Pipeline
replacing_pipeline = Pipeline([           
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="constant", fill_value=0)),
    ("replacements", replaceVals()),
])

# Combined Pipeline
full_pipeline = ColumnTransformer([
    ("cat", OneHotEncoder(), one_hot_cols),
    ("num", replacing_pipeline, fill_cols),
])

## Transform data
We will pass our data through the pipeline and then split into training and test splits. We can also try to do stratified sampling.

In [0]:
prepped_data = full_pipeline.fit_transform(dropout)
# Split labels
prepped_x = prepped_data[:,:16]
prepped_y = prepped_data[:,16]
print(prepped_x.shape, prepped_y.shape)

(100046, 16) (100046,)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(prepped_x, prepped_y, test_size=0.2, random_state=42)
X_train=X_train.astype("float64")
X_test=X_test.astype("float64")
y_train=y_train.astype("float64")
y_test=y_test.astype("float64")

# Testing Models
We will test the following regressors:

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor

## Linear Regression
We will do standard linear regression as a baseline, and we don't really need to worry about hyperparameter tuning (as there aren't any hyperparameters to tune)

In [0]:
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lin_model, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

Scores: [147.59398266 157.30600806 162.55301942 142.54374353 159.12090589
 213.21836246 122.13159037 114.15308042 142.40040326 169.94796839]
Mean: 153.09690644538782
Standard deviation: 26.025967653694522


## Random Forest
Here are the parameters we will test

In [0]:
n_estimators = [10, 50, 100, 250, 375, 500, 750, 1000]
max_features = ['auto', 'sqrt']
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]# Create the random grid

param_distribs = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

**Randomized Search**:

In [0]:
from sklearn.model_selection import RandomizedSearchCV

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of ? rounds of training 
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=5, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


**Get the best model:**

In [0]:
final_rf_model = rnd_search.best_estimator_
rnd_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=110,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=750,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

**Test model**:

In [0]:
scores = cross_val_score(final_rf_model, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

## SGD Regression
Here is the parameter grid

In [0]:
loss = ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
penalty = ['l1', 'l2', 'elasticnet']
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive']

param_grid = {'loss': loss,
              'penalty': penalty,
              'alpha': alpha,
              'learning_rate': learning_rate}

**Randomized Search**:

In [0]:
sgd_model = SGDRegressor(random_state=42)
# train across 5 folds, that's a total of ? rounds of training 
rnd_search = RandomizedSearchCV(sgd_model, param_distributions=param_grid,
                                n_iter=25, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=SGDRegressor(alpha=0.0001, average=False,
                                          early_stopping=False, epsilon=0.1,
                                          eta0=0.01, fit_intercept=True,
                                          l1_ratio=0.15,
                                          learning_rate='invscaling',
                                          loss='squared_loss', max_iter=1000,
                                          n_iter_no_change=5, penalty='l2',
                                          power_t=0.25, random_state=42,
                                          shuffle=True, tol=0.001,
                                          validation_fraction=0.1...
                   param_distributions={'alpha': [0.0001, 0.001, 0.01, 0.1, 1,
                                                  10, 100, 1000],
                                        'learning_rate': ['constant', 'optimal',
     

**Get the best model:**

In [0]:
final_sgd_model = rnd_search.best_estimator_
rnd_search.best_estimator_

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='adaptive', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l1', power_t=0.25,
             random_state=42, shuffle=True, tol=0.001, validation_fraction=0.1,
             verbose=0, warm_start=False)

**Test model**:

In [0]:
scores = cross_val_score(final_sgd_model, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

Scores: [170.93236382 127.40438355 124.38806859 133.37693297 171.08313907
 163.30791797 188.34013952 129.18103328 148.32006898 198.86141671]
Mean: 155.5195464468873
Standard deviation: 25.47866659560963


---
## Evaluating Models

In [0]:
# Here are our models
models = [lin_model, final_rf_model, final_sgd_model]
d = {0: "Linear Regression", 1: "Random Forest", 2: "SGD Regression"}
# Our confidence level
confidence = 0.95

In [0]:
from sklearn.metrics import mean_squared_error
from scipy import stats

for i, model in enumerate(models):
  # Get predictions and mse
  final_predictions = model.predict(X_test)
  final_mse = mean_squared_error(y_test, final_predictions)
  # Get the rmse
  final_rmse = np.sqrt(final_mse)
  
  # Calculate moe
  squared_errors = (final_predictions - y_test) ** 2
  moe = np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))  

  print(d[i], final_rmse, moe)
  

Linear Regression 186.54066044922365 [ 78.11076267 251.97925462]
Random Forest 65.5707203293066 [        nan 96.65891244]
SGD Regression 194.60221215349387 [ 85.35312273 261.63884725]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in sqrt
  del sys.path[0]


---

# KMEANS Testing